In [3]:
import requests

def query_model(prompt , number_responses=1):
    url = 'http://127.0.0.1:5000/generate'
    
    response = requests.post(url, json={'prompt': prompt, 'number_responses' : number_responses })
    if response.status_code == 200:
        return response.json().get('responses')
    else:
        return "Error: " + response.text

# Example usage
initial_system ="Background information or context: " + 'You are a reward engineer trying to write reward functions to solve reinforcement learning tasks as effective as possible.\nYour goal is to write a reward function for the environment that will help the agent learn the task described in text. \nYour reward function should use useful variables from the environment as inputs. As an example,\nthe reward function signature can be: @torch.jit.script\ndef compute_reward(object_pos: torch.Tensor, goal_pos: torch.Tensor) -> Tuple[torch.Tensor, Dict[str, torch.Tensor]]:\n    ...\n    return reward, {}\n\nSince the reward function will be decorated with @torch.jit.script,\nplease make sure that the code is compatible with TorchScript (e.g., use torch tensor instead of numpy array). \nMake sure any new tensor or variable you introduce is on the same device as the input tensors. The output of the reward function should consist of two items:\n    (1) the total reward,\n    (2) a dictionary of each individual reward component.\nThe code output should be formatted as a python code string: "```python ... ```".\n\nSome helpful tips for writing the reward function code:\n    (1) You may find it helpful to normalize the reward to a fixed range by applying transformations like torch.exp to the overall reward or its components\n    (2) If you choose to transform a reward component, then you must also introduce a temperature parameter inside the transformation function; this parameter must be a named variable in the reward function and it must not be an input variable. Each transformed reward component should have its own temperature variable\n    (3) Make sure the type of each input variable is correctly specified; a float input variable should not be specified as torch.Tensor\n    (4) Most importantly, the reward code\'s input variables must contain only attributes of the provided environment class definition (namely, variables that have prefix self.). Under no circumstance can you introduce new input variables.\n'
initial_user = 'The Python environment is class ShadowHand(VecTask):\n    """Rest of the environment definition omitted."""\n    def compute_observations(self):\n        self.gym.refresh_dof_state_tensor(self.sim)\n        self.gym.refresh_actor_root_state_tensor(self.sim)\n        self.gym.refresh_rigid_body_state_tensor(self.sim)\n\n        if self.obs_type == "full_state" or self.asymmetric_obs:\n            self.gym.refresh_force_sensor_tensor(self.sim)\n            self.gym.refresh_dof_force_tensor(self.sim)\n\n        self.object_pose = self.root_state_tensor[self.object_indices, 0:7]\n        self.object_pos = self.root_state_tensor[self.object_indices, 0:3]\n        self.object_rot = self.root_state_tensor[self.object_indices, 3:7]\n        self.object_linvel = self.root_state_tensor[self.object_indices, 7:10]\n        self.object_angvel = self.root_state_tensor[self.object_indices, 10:13]\n\n        self.goal_pose = self.goal_states[:, 0:7]\n        self.goal_pos = self.goal_states[:, 0:3]\n        self.goal_rot = self.goal_states[:, 3:7]\n\n        self.fingertip_state = self.rigid_body_states[:, self.fingertip_handles][:, :, 0:13]\n        self.fingertip_pos = self.rigid_body_states[:, self.fingertip_handles][:, :, 0:3]\n\n        if self.obs_type == "openai":\n            self.compute_fingertip_observations(True)\n        elif self.obs_type == "full_no_vel":\n            self.compute_full_observations(True)\n        elif self.obs_type == "full":\n            self.compute_full_observations()\n        elif self.obs_type == "full_state":\n            self.compute_full_state()\n        else:\n            print("Unknown observations type!")\n\n        if self.asymmetric_obs:\n            self.compute_full_state(True)\n\n    def compute_full_state(self, asymm_obs=False):\n        if asymm_obs:\n            self.states_buf[:, 0:self.num_shadow_hand_dofs] = unscale(self.shadow_hand_dof_pos,\n                                                                      self.shadow_hand_dof_lower_limits, self.shadow_hand_dof_upper_limits)\n            self.states_buf[:, self.num_shadow_hand_dofs:2*self.num_shadow_hand_dofs] = self.vel_obs_scale * self.shadow_hand_dof_vel\n            self.states_buf[:, 2*self.num_shadow_hand_dofs:3*self.num_shadow_hand_dofs] = self.force_torque_obs_scale * self.dof_force_tensor\n\n            obj_obs_start = 3*self.num_shadow_hand_dofs  # 72\n            self.states_buf[:, obj_obs_start:obj_obs_start + 7] = self.object_pose\n            self.states_buf[:, obj_obs_start + 7:obj_obs_start + 10] = self.object_linvel\n            self.states_buf[:, obj_obs_start + 10:obj_obs_start + 13] = self.vel_obs_scale * self.object_angvel\n\n            goal_obs_start = obj_obs_start + 13  # 85\n            self.states_buf[:, goal_obs_start:goal_obs_start + 7] = self.goal_pose\n            self.states_buf[:, goal_obs_start + 7:goal_obs_start + 11] = quat_mul(self.object_rot, quat_conjugate(self.goal_rot))\n\n            num_ft_states = 13 * self.num_fingertips  # 65\n            num_ft_force_torques = 6 * self.num_fingertips  # 30\n\n            fingertip_obs_start = goal_obs_start + 11  # 96\n            self.states_buf[:, fingertip_obs_start:fingertip_obs_start + num_ft_states] = self.fingertip_state.reshape(self.num_envs, num_ft_states)\n            self.states_buf[:, fingertip_obs_start + num_ft_states:fingertip_obs_start + num_ft_states +\n                            num_ft_force_torques] = self.force_torque_obs_scale * self.vec_sensor_tensor\n\n            obs_end = fingertip_obs_start + num_ft_states + num_ft_force_torques\n            self.states_buf[:, obs_end:obs_end + self.num_actions] = self.actions\n        else:\n            self.obs_buf[:, 0:self.num_shadow_hand_dofs] = unscale(self.shadow_hand_dof_pos,\n                                                                   self.shadow_hand_dof_lower_limits, self.shadow_hand_dof_upper_limits)\n            self.obs_buf[:, self.num_shadow_hand_dofs:2*self.num_shadow_hand_dofs] = self.vel_obs_scale * self.shadow_hand_dof_vel\n            self.obs_buf[:, 2*self.num_shadow_hand_dofs:3*self.num_shadow_hand_dofs] = self.force_torque_obs_scale * self.dof_force_tensor\n\n            obj_obs_start = 3*self.num_shadow_hand_dofs  # 72\n            self.obs_buf[:, obj_obs_start:obj_obs_start + 7] = self.object_pose\n            self.obs_buf[:, obj_obs_start + 7:obj_obs_start + 10] = self.object_linvel\n            self.obs_buf[:, obj_obs_start + 10:obj_obs_start + 13] = self.vel_obs_scale * self.object_angvel\n\n            goal_obs_start = obj_obs_start + 13  # 85\n            self.obs_buf[:, goal_obs_start:goal_obs_start + 7] = self.goal_pose\n            self.obs_buf[:, goal_obs_start + 7:goal_obs_start + 11] = quat_mul(self.object_rot, quat_conjugate(self.goal_rot))\n\n            num_ft_states = 13 * self.num_fingertips  # 65\n            num_ft_force_torques = 6 * self.num_fingertips  # 30\n\n            fingertip_obs_start = goal_obs_start + 11  # 96\n            self.obs_buf[:, fingertip_obs_start:fingertip_obs_start + num_ft_states] = self.fingertip_state.reshape(self.num_envs, num_ft_states)\n            self.obs_buf[:, fingertip_obs_start + num_ft_states:fingertip_obs_start + num_ft_states +\n                         num_ft_force_torques] = self.force_torque_obs_scale * self.vec_sensor_tensor\n\n            obs_end = fingertip_obs_start + num_ft_states + num_ft_force_torques\n            self.obs_buf[:, obs_end:obs_end + self.num_actions] = self.actions\n. Write a reward function for the following task: to make the shadow hand spin the object to a target orientation.'
combined_user_message = initial_system + "\n" + initial_user
er= query_model(combined_user_message, 2)

In [4]:
print(er)

['[INST] Background information or context: You are a reward engineer trying to write reward functions to solve reinforcement learning tasks as effective as possible.\nYour goal is to write a reward function for the environment that will help the agent learn the task described in text. \nYour reward function should use useful variables from the environment as inputs. As an example,\nthe reward function signature can be: @torch.jit.script\ndef compute_reward(object_pos: torch.Tensor, goal_pos: torch.Tensor) -> Tuple[torch.Tensor, Dict[str, torch.Tensor]]:\n    ...\n    return reward, {}\n\nSince the reward function will be decorated with @torch.jit.script,\nplease make sure that the code is compatible with TorchScript (e.g., use torch tensor instead of numpy array). \nMake sure any new tensor or variable you introduce is on the same device as the input tensors. The output of the reward function should consist of two items:\n    (1) the total reward,\n    (2) a dictionary of each individ

In [3]:
er